# Proyecto 1 - Visión por Computadora
## Ejercicio 2
## Integrantes:

- Javier Alvarado - 21188
- Mario Guerra - 21008
- Emilio Solano - 21212

In [15]:
import cv2
import numpy as np
import networkx as nex
import json
import os
from skimage.morphology import skeletonize
from scipy import ndimage


def load_image(image_path):
    """
    Carga una imagen groundtruth y la convierte a binaria (0/1).
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {image_path}")
    _, binary_img = cv2.threshold(img, 127, 1, cv2.THRESH_BINARY)
    return binary_img

def get_skeleton(binary_img):
    """
    Obtiene el esqueleto de la imagen binaria usando skeletonize de scikit-image.
    Retorna un array binario (0/1).
    """
    skeleton_bool = skeletonize(binary_img == 1)
    skeleton_bin = skeleton_bool.astype(np.uint8)
    return skeleton_bin

def classify_node(neighbor_count):
    """
    Clasifica un nodo (píxel en el esqueleto) según su número de vecinos (en 8 direcciones).
    """
    if neighbor_count == 1:
        return "extremo"       # (verde en la figura)
    elif neighbor_count == 2:
        return "intermedio"    # (gris en la figura)
    elif neighbor_count == 3:
        return "bifurcacion"   # (rojo en la figura)
    elif neighbor_count >= 4:
        return "trifurcacion"  # (azul en la figura)
    else:
        # neighbor_count == 0 sucede si algo sale mal, o no es parte del esqueleto.
        return "desconocido"

def find_8_neighbors(y, x, shape):
    """
    Retorna la lista de coordenadas (y2,x2) que son vecinos en 8 direcciones
    de (y,x), dentro de los límites de la imagen.
    """
    (h, w) = shape
    neighbors = []
    for dy in [-1, 0, 1]:
        for dx in [-1, 0, 1]:
            if dy == 0 and dx == 0:
                continue
            ny, nx = y + dy, x + dx
            if 0 <= ny < h and 0 <= nx < w:
                neighbors.append((ny, nx))
    return neighbors

In [16]:
def build_graph_with_classification(skeleton_bin):
    """
    Construye un grafo donde cada píxel del esqueleto es un nodo.
    Clasifica cada nodo en 'extremo', 'bifurcacion', 'trifurcacion' o 'intermedio'.
    Crea una arista (u,v) para cada par de nodos adyacentes (en 8-direcciones) del esqueleto.
    
    Retorna (G, dict_lists) donde:
      - G es un nx.Graph con atributos de cada nodo.
      - dict_lists es un diccionario con las listas separadas de nodos y la lista de aristas.
    """
    # Dimensiones
    h, w = skeleton_bin.shape
    
    # Paso 1: recolectar coordenadas de todos los píxeles que pertenezcan al esqueleto
    skeleton_pixels = np.argwhere(skeleton_bin == 1)
    
    # Para mapear coordenadas -> id de nodo y viceversa
    coord_to_id = {}
    for i, (y, x) in enumerate(skeleton_pixels):
        coord_to_id[(y, x)] = i
    
    # Crear grafo
    G = nex.Graph()
    
    # Paso 2: Contar vecinos de cada píxel y clasificar
    kernel = np.ones((3,3), dtype=np.uint8)
    kernel[1,1] = 0
    neighbors_count = ndimage.convolve(skeleton_bin, kernel, mode='constant', cval=0)

    for (y, x) in skeleton_pixels:
        num_neighbors = neighbors_count[y, x]
        node_type = classify_node(num_neighbors)
        node_id = coord_to_id[(y, x)]
        G.add_node(node_id, 
                   pos=(int(y), int(x)),
                   tipo=node_type)
    
    # Paso 3: Crear aristas para cada par de píxeles vecinos en 8 direcciones
    for (y, x) in skeleton_pixels:
        node_id = coord_to_id[(y, x)]
        neighbors_8 = find_8_neighbors(y, x, (h, w))
        for (ny, nx) in neighbors_8:
            if skeleton_bin[ny, nx] == 1:
                neighbor_id = coord_to_id[(ny, nx)]
                if neighbor_id > node_id:
                    G.add_edge(node_id, neighbor_id)
    
    # Ahora creamos un diccionario con las listas solicitadas.
    end_nodes = []
    bifurcation_nodes = []
    trifurcation_nodes = []
    intermediate_nodes = []

    for n in G.nodes():
        node_type = G.nodes[n]['tipo']
        (y, x) = G.nodes[n]['pos']
        if node_type == "extremo":
            end_nodes.append({"id": n, "fila": y, "columna": x})
        elif node_type == "bifurcacion":
            bifurcation_nodes.append({"id": n, "fila": y, "columna": x})
        elif node_type == "trifurcacion":
            trifurcation_nodes.append({"id": n, "fila": y, "columna": x})
        elif node_type == "intermedio":
            intermediate_nodes.append({"id": n, "fila": y, "columna": x})
    
    # Listado de aristas (en amarillo en la figura).
    # Cada arista es un par (u,v). Opcionalmente podríamos guardar la lista de
    # píxeles entre medio, pero aquí cada arista sólo conecta píxeles adyacentes.
    edges = []
    for (u, v) in G.edges():
        edges.append({"origen": u, "destino": v})

    dict_lists = {
        "nodos_extremos" : end_nodes,
        "nodos_bifurcacion": bifurcation_nodes,
        "nodos_trifurcacion": trifurcation_nodes,
        "nodos_intermedios": intermediate_nodes,
        "aristas": edges
    }
    
    return G, dict_lists

def save_graph_json(dict_lists, output_path):
    """
    Guarda en un JSON las listas solicitadas:
    - nodos_extremos, nodos_bifurcacion, nodos_trifurcacion, nodos_intermedios
    - aristas
    """
    with open(output_path, 'w') as f:
        json.dump(dict_lists, f, indent=4)
    print(f"Guardado JSON en {output_path}")

def visualize_classified_graph(skeleton_bin, G, output_path=None):
    """
    Visualiza el grafo sobre la imagen. Se colorean los nodos según su tipo:
        - Extremo (verde)
        - Bifurcación (rojo)
        - Trifurcación (azul)
        - Intermedio (gris)
    Y las aristas se dibujan en amarillo.
    """
    # Pasar a BGR para dibujar en color
    # Multiplicamos por 255 para que se vea "blanco" el esqueleto original en la visualización.
    img_color = cv2.cvtColor(skeleton_bin*255, cv2.COLOR_GRAY2BGR)

    # Dibujamos las aristas en amarillo
    for (u, v) in G.edges():
        y1, x1 = G.nodes[u]['pos']
        y2, x2 = G.nodes[v]['pos']
        # Amarillo = (0,255,255) en BGR
        cv2.line(img_color, (x1,y1), (x2,y2), (0,255,255), 1)
    
    # Dibujamos los nodos según su tipo
    for n in G.nodes():
        y, x = G.nodes[n]['pos']
        node_type = G.nodes[n]['tipo']
        if node_type == "extremo":
            color = (0, 255, 0)    # verde
        elif node_type == "bifurcacion":
            color = (0, 0, 255)    # rojo
        elif node_type == "trifurcacion":
            color = (255, 0, 0)    # azul
        else:
            color = (128, 128, 128) # gris
        cv2.circle(img_color, (x, y), 2, color, -1)

    if output_path:
        cv2.imwrite(output_path, img_color)
        print(f"Visualización guardada en {output_path}")
    
    return img_color

def process_image(image_path, output_json_path, output_vis_path=None):
    """
    Procesa una imagen y genera el grafo con la clasificación de nodos.
    """
    # 1) Cargar y esqueletizar
    binary_img = load_image(image_path)
    skeleton_bin = get_skeleton(binary_img)
    
    # 2) Construir grafo con clasificación
    G, dict_lists = build_graph_with_classification(skeleton_bin)
    
    # 3) Guardar JSON
    save_graph_json(dict_lists, output_json_path)
    
    # 4) Visualizar (opcional)
    if output_vis_path:
        visualize_classified_graph(skeleton_bin, G, output_vis_path)
    
    return G, dict_lists

def process_all_images(input_dir, output_dir):
    """
    Procesa todas las imágenes groundtruth en el directorio especificado.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i in range(1, 21):
        file_name = f"{i}_gt.pgm"
        image_path = os.path.join(input_dir, file_name)
        
        if os.path.exists(image_path):
            print(f"Procesando {file_name}...")
            output_json_path = os.path.join(output_dir, f"{i}_grafo.json")
            output_vis_path = os.path.join(output_dir, f"{i}_visualizacion.png")
            process_image(image_path, output_json_path, output_vis_path)
        else:
            print(f"No se encontró el archivo {image_path}")

In [17]:
data = "./data/database/"
out_directory = "./artery-discretization/"
process_all_images(data, out_directory)

Procesando 1_gt.pgm...
Guardado JSON en ./artery-discretization/1_grafo.json
Visualización guardada en ./artery-discretization/1_visualizacion.png
Procesando 2_gt.pgm...
Guardado JSON en ./artery-discretization/2_grafo.json
Visualización guardada en ./artery-discretization/2_visualizacion.png
Procesando 3_gt.pgm...
Guardado JSON en ./artery-discretization/3_grafo.json
Visualización guardada en ./artery-discretization/3_visualizacion.png
Procesando 4_gt.pgm...
Guardado JSON en ./artery-discretization/4_grafo.json
Visualización guardada en ./artery-discretization/4_visualizacion.png
Procesando 5_gt.pgm...
Guardado JSON en ./artery-discretization/5_grafo.json
Visualización guardada en ./artery-discretization/5_visualizacion.png
Procesando 6_gt.pgm...
Guardado JSON en ./artery-discretization/6_grafo.json
Visualización guardada en ./artery-discretization/6_visualizacion.png
Procesando 7_gt.pgm...
Guardado JSON en ./artery-discretization/7_grafo.json
Visualización guardada en ./artery-discre